<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/FinRL_ensemble_stock_trading_ICAIF_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading Using Ensemble Strategy

Tutorials to use OpenAI DRL to trade multiple stocks using ensemble strategy in one Jupyter Notebook | Presented at ICAIF 2020

* This notebook is the reimplementation of our paper: Deep Reinforcement Learning for Automated Stock Trading: An Ensemble Strategy, using FinRL.
* Check out medium blog for detailed explanations: https://medium.com/@ai4finance/deep-reinforcement-learning-for-automated-stock-trading-f1dad0126a02
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [1]:
# ## install finrl library
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent,DRLEnsembleAgent
# from finrl.trade.backtest import get_baseline, backtest_stats, backtest_plot

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

KeyError: 'A2C2_PARAMS'

<a id='1.4'></a>
## 2.4. Create Folders

In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [3]:
# from config.py start_date is a string
config.START_DATE

'2004-08-11'

In [4]:
print(config.DOW_30_TICKER)

['AAPL', 'MSFT', 'JPM', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD', 'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD', 'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
# from finrl.config.config import MISSING3
# df = YahooDownloader(start_date = '2006-01-01',
#                      end_date = '2021-06-11',
#                      ticker_list = config.DOW_30_TICKER).fetch_data()

In [7]:
# df.append(df2).append(df3).append(df4).append(df5)

In [8]:
import pickle
# df.drop('Unnamed: 0')with open('dji_df_2004-2021.pkl', 'rb') as f:
#     df = pickle.load(f)

df = pd.read_csv('dji_prices_2020_04_09.csv')
df

,Unnamed: 0,date,open,high,low,close,volume,tic,day
0,0,2006-01-03,2.585000,2.669643,2.580357,2.295634,807234400,AAPL,1
1,1,2006-01-03,51.700001,52.580002,51.049999,41.155041,7825700,AXP,1
2,2,2006-01-03,70.400002,70.599998,69.330002,50.119705,4943000,BA,1
3,3,2006-01-03,57.869999,58.110001,57.049999,38.086823,3697500,CAT,1
4,4,2006-01-03,17.209999,17.490000,17.180000,12.956775,55426000,CSCO,1
...,...,...,...,...,...,...,...,...,...
116020,116020,2021-06-10,233.100006,234.259995,232.130005,233.949997,4452500,V,3
116021,116021,2021-06-10,57.330002,57.610001,57.220001,57.340000,12013600,VZ,3
116022,116022,2021-06-10,53.799999,55.580002,53.570000,55.310001,6638000,WBA,3
116023,116023,2021-06-10,139.080002,140.190002,139.080002,139.880005,5459500,WMT,3


In [9]:
df = df.drop(['Unnamed: 0'], axis=1)

In [10]:
df = df.sort_values(['date', 'tic'])

In [11]:
df.head()

,date,open,high,low,close,volume,tic,day
0,2006-01-03,2.585000,2.669643,2.580357,2.295634,807234400,AAPL,1
1,2006-01-03,51.700001,52.580002,51.049999,41.155041,7825700,AXP,1
2,2006-01-03,70.400002,70.599998,69.330002,50.119705,4943000,BA,1
3,2006-01-03,57.869999,58.110001,57.049999,38.086823,3697500,CAT,1
4,2006-01-03,17.209999,17.490000,17.180000,12.956775,55426000,CSCO,1


In [12]:
df.tail()

,date,open,high,low,close,volume,tic,day
116020,2021-06-10,233.100006,234.259995,232.130005,233.949997,4452500,V,3
116021,2021-06-10,57.330002,57.610001,57.220001,57.340000,12013600,VZ,3
116022,2021-06-10,53.799999,55.580002,53.570000,55.310001,6638000,WBA,3
116023,2021-06-10,139.080002,140.190002,139.080002,139.880005,5459500,WMT,3
116024,2021-06-10,63.610001,63.980000,62.250000,62.750000,27488700,XOM,3


In [13]:
df.shape

(116025, 8)

In [14]:
df.tic.unique()

array(['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GS', 'HD',
       'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT',
       'NKE', 'PFE', 'PG', 'RTX', 'TRV', 'UNH', 'VZ', 'WBA', 'WMT', 'XOM',
       'V'], dtype=object)

In [15]:
# import pickle
# with open('dji_df_2004-2021.pkl', 'wb') as f:
#     pickle.dump(df, f)

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [16]:
import pickle
with open('/home/roman/Work/trading-bot/notebooks/2004-2021_imputed_sentiment_final_df.pkl', 'rb') as f:
    daily_sentiment_df = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/home/roman/Work/trading-bot/notebooks/2004-2021_imputed_sentiment_final_df.pkl'

In [17]:
with open('doc2vec_2004_2021_expanded_world_df.pkl', 'rb') as f:
    doc2vec_2004_2021_expanded_world_df = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'doc2vec_2004_2021_expanded_world_df.pkl'

In [18]:
daily_sentiment_df

NameError: name 'daily_sentiment_df' is not defined

In [19]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=True)

processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [20]:
list(processed.columns)

['date',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'tic',
 'day',
 'macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma',
 'turbulence']

In [21]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

NameError: name 'itertools' is not defined

In [ ]:
processed_full.sample(5)

In [ ]:
processed_full['date'][0]

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

In [ ]:
config.TECHNICAL_INDICATORS_LIST

In [ ]:
stock_dimension = len(processed_full.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, User Features: {config.NUMBER_OF_USER_FEATURES}, State Space: {state_space}")


In [ ]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 50_000_000/100, #Since in Indonesia the minimum number of shares per trx is 100, then we scaled the initial amount by dividing it with 100 
    "buy_cost_pct": 0.001, #IPOT has 0.19% buy cost
    "sell_cost_pct": 0.001, #IPOT has 0.29% sell cost
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":5
    
}

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

* In this notebook, we are training and validating 3 agents (A2C, PPO, DDPG) using Rolling-window Ensemble Method ([reference code](https://github.com/AI4Finance-LLC/Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020/blob/80415db8fa7b2179df6bd7e81ce4fe8dbf913806/model/models.py#L92))

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
rebalance_window = 63 # rebalance_window is the number of days to retrain the model
validation_window = 63 # validation_window is the number of days to do validation and trading (e.g. if validation_window=63, then both validation and trading period will be 63 days)
train_start = '2006-01-03'
train_end = '2016-01-01'
val_test_start = '2016-01-01'
val_test_end = '2021-06-11'

ensemble_agent = DRLEnsembleAgent(df=processed_full,
                 train_period=(train_start,train_end),
                 val_test_period=(val_test_start,val_test_end),
                 rebalance_window=rebalance_window, 
                 validation_window=validation_window,
                 **env_kwargs)

In [32]:
print([0]*0)

[]


In [ ]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.01,
                    'learning_rate': 0.0005
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      "action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 50_000,
                      "learning_rate": 0.000005,
                      "batch_size": 128
                    }

A2C2_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.01,
                    'learning_rate': 0.001
                    }

PPO2_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.0005,
                    "batch_size": 256
                    }

DDPG2_model_kwargs = {
                      "action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 50_000,
                      "learning_rate": 0.00001,
                      "batch_size": 256
                    }

timesteps_dict = {'a2c' : 25_000, 
                 'ppo' : 25_000, 
                 'ddpg' : 15_000,
                  'a2c2' : 15_000, 
                 'ppo2' : 15_000, 
                 'ddpg2' : 10_000
                 }

In [ ]:
import time
start = time.time()# print([self.state[0]])
        # print(self.data.close.values.tolist())
        # print(list(self.state[(self.stock_dim+1):(self.stock_dim*2+1)]))
        # print(sum([self.data[tech].values.tolist() for tech in self.tech_indicator_list ], []) )
        # user_features_columns = self.data.columns[-config.NUMBER_OF_USER_FEATURES:]
        # print(self.data[user_features_columns].values[0])
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs, A2C2_model_kwargs, PPO_model_kwargs, PPO2_model_kwargs, DDPG_model_kwargs, DDPG2_model_kwargs, timesteps_dict)
time_elapsed = time.time()-start
print(time_elapsed)

In [33]:
df_summary

,Iter,Val Start,Val End,Model Used,A2C Sharpe,PPO Sharpe,DDPG Sharpe
0,126,2016-01-04,2016-04-05,A2C,0.2314,0.137868,0.138116
1,189,2016-04-05,2016-07-05,DDPG,0.023251,-0.02747,0.027674
2,252,2016-07-05,2016-10-03,DDPG,0.110485,-0.199811,0.160881
3,315,2016-10-03,2017-01-03,DDPG,0.314206,0.153133,0.621001
4,378,2017-01-03,2017-04-04,DDPG,0.203685,0.021583,0.21147
5,441,2017-04-04,2017-07-05,DDPG,0.095114,0.033792,0.317426
6,504,2017-07-05,2017-10-03,A2C,0.377324,0.121963,0.251148
7,567,2017-10-03,2018-01-03,DDPG,0.574725,0.341799,0.793806
8,630,2018-01-03,2018-04-05,A2C,0.001569,-0.112015,-0.014637
9,693,2018-04-05,2018-07-05,DDPG,-0.02719,-0.1257,-0.015095


In [ ]:
del fe

In [34]:
import dill
dill.dump_session('17_year_sharpe_reward_dow.db')

In [35]:
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2006-01-03,AAPL,2.585000,2.669643,2.580357,2.295634,807234400.0,1.0,0.000000,2.308567,2.289456,100.000000,66.666667,100.000000,2.295634,2.295634,0.000000
1,2006-01-03,AXP,51.700001,52.580002,51.049999,41.155041,7825700.0,1.0,0.000000,2.308567,2.289456,100.000000,66.666667,100.000000,41.155041,41.155041,0.000000
2,2006-01-03,BA,70.400002,70.599998,69.330002,50.119705,4943000.0,1.0,0.000000,2.308567,2.289456,100.000000,66.666667,100.000000,50.119705,50.119705,0.000000
3,2006-01-03,CAT,57.869999,58.110001,57.049999,38.086823,3697500.0,1.0,0.000000,2.308567,2.289456,100.000000,66.666667,100.000000,38.086823,38.086823,0.000000
4,2006-01-03,CSCO,17.209999,17.490000,17.180000,12.956775,55426000.0,1.0,0.000000,2.308567,2.289456,100.000000,66.666667,100.000000,12.956775,12.956775,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169139,2021-06-10,V,233.100006,234.259995,232.130005,233.949997,4452500.0,3.0,1.668748,233.608472,222.814534,58.710599,108.800722,30.377055,228.689480,224.125779,33.831745
169135,2021-06-10,VZ,57.330002,57.610001,57.220001,57.340000,12013600.0,3.0,-0.144320,58.482470,55.706530,50.785631,-13.987824,0.130661,57.556333,57.430342,33.831745
169136,2021-06-10,WBA,53.799999,55.580002,53.570000,55.310001,6638000.0,3.0,0.242677,55.862549,52.305305,56.676743,75.341122,1.054941,53.905892,53.519815,33.831745
169137,2021-06-10,WMT,139.080002,140.190002,139.080002,139.880005,5459500.0,3.0,0.202702,143.623730,138.455272,50.613731,-58.482475,8.501810,140.619338,138.698263,33.831745


In [36]:
import pickle
with open('17_year_sharpe_reward_dow_df_summary.pkl', 'wb') as f:
    pickle.dump(df_summary, f)

In [37]:
with open('17_year_sharpe_reward_dow.pkl_processed_full', 'wb') as f:
    pickle.dump(processed_full, f)

In [38]:
with open('17_year_sharpe_reward_dow_agent.pkl', 'wb') as f:
    pickle.dump(ensemble_agent, f)